In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Colab Notebooks/Cross Modal Retrieval'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/Cross Modal Retrieval


In [2]:
!pip install open3d
!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.5.0.html

     |████████████████████████████████| 4.6MB 8.5MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 153kB 8.2MB/s 
     |████████████████████████████████| 24.4MB 116kB/s 
     |████████████████████████████████| 12.3MB 60.6MB/s 
     |████████████████████████████████| 18.2MB 169kB/s 
     |████████████████████████████████| 235kB 18.9MB/s 
     |████████████████████████████████| 2.1MB 20.6MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.5.0-cp36-none-any.whl size=267918 sha256=97fc5144beb3d97a39461a42b47e86fe3c6b631c2fbbc64dd66487923ef5b930
  Stored in directory: /root/.cache/pip/wheels/ec/51/31/5786f2ac419ee312f22d4d2877da05f20e7f2d430e22917daf
Successfully built torch-geometric


In [3]:
from text_extraction_torch import get_text_embedding
import open3d as o3d
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

embedding_matrix, encoder_input_sentences,modelIdList,categoryList = get_text_embedding("descriptions.csv")
print("embedding_matrix:",embedding_matrix.shape,"sentences:",encoder_input_sentences.shape,"modelIdList:",len(modelIdList))

Chair/2b110b833111b38c420adf24e49f74c8.ply does not exist
Chair/bd6bc40b3327e5109501f2fa226917d0.ply does not exist
Chair/1d498876c8890f7786470a1318504fef.ply does not exist
Chair/8979c1aaa6675009bf80985a99195eb8.ply does not exist
Chair/e4b40369894a16ce6821a1e68ba5ebab.ply does not exist
Chair/311c07b96ce39f82c70d3fe6f2ca143f.ply does not exist
Chair/bdd57499bf64fab6bf80985a99195eb8.ply does not exist
Chair/5ce323f8499cf60ce51e3cd2b72fbf02.ply does not exist
Chair/9ee4b9c97bcf4b3715dec43ae6a12831.ply does not exist
Chair/c9288e09458ec0e9bbc45ba666751519.ply does not exist
Chair/48429b3467c7185060fcaed6cc231482.ply does not exist
Chair/5afc95ed2a26ce548644074928ab3f32.ply does not exist
Chair/475cf49d3764f04ecc8bd7a04c9659f1.ply does not exist
Chair/e4b40369894a16ce6821a1e68ba5ebab.ply does not exist
Chair/bdd57499bf64fab6bf80985a99195eb8.ply does not exist
Table/10cd74fb0af8021fbf7cf6012a0af9fc.ply does not exist
Chair/e6a5c9fdb609d00a5ff6fe84f53e00a5.ply does not exist
Chair/64198879

In [0]:
class ShapeText(Dataset):
    def __init__(self, encoder_input_sentences,modelIdList, categoryList, train=True, transform=None):
        self.is_train = train
        self.transform = transform
        self.encoder_input_sentences = encoder_input_sentences
        self.files = []
        for idx,modelId in enumerate(modelIdList):
            sample = {}
            sample['pcd_path'] = os.path.join(categoryList[idx], modelId + ".ply")
            self.files.append(sample)
          
    
    def __len__(self):
        return len(self.files)
    
    def __preproc__(self, pcd_load):
        # convert Open3D.o3d.geometry.PointCloud to numpy array
        xyz_load = np.asarray(pcd_load.points)
        color_load = np.asarray(pcd_load.colors)
        if self.transform:
            xyz = self.transform(xyz_load)
        return xyz,color_load

    def __getitem__(self, idx):
        pcd_path = self.files[idx]['pcd_path']
        pcd_load = o3d.io.read_point_cloud(pcd_path)
        xyz, color = self.__preproc__(pcd_load)
        pos_desc = self.encoder_input_sentences[idx]
        if not self.is_train:
          return {'xyz': xyz, 'color': color}, pos_desc
        negative_idx = random.choice([ i for i in range(len(self.files)) if (i != idx and modelIdList[i]!=modelIdList[idx])])
        neg_desc = self.encoder_input_sentences[negative_idx]
        return {'xyz': xyz, 'color': color},pos_desc, neg_desc

In [0]:
from torch import nn
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
        
    def calc_euclidean(self, x1, x2):
        return (x1 - x2).pow(2).sum(1)
    
    def forward(self, anchor: torch.Tensor, positive: torch.Tensor, negative: torch.Tensor) -> torch.Tensor:
        distance_positive = self.calc_euclidean(anchor, positive)
        distance_negative = self.calc_euclidean(anchor, negative)
        losses = torch.relu(distance_positive - distance_negative + self.margin)

        return losses.mean()

In [0]:
import open3d as o3d
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

class Normalize(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        
        norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0)
        norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

        return  norm_pointcloud

class RandRotation_z(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        theta = random.random() * 2. * math.pi
        rot_matrix = np.array([[ math.cos(theta), -math.sin(theta),    0],
                               [ math.sin(theta),  math.cos(theta),    0],
                               [0,                             0,      1]])
        
        rot_pointcloud = rot_matrix.dot(pointcloud.T).T
        return  rot_pointcloud
    
class RandomNoise(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        noise = np.random.normal(0, 0.02, (pointcloud.shape))
    
        noisy_pointcloud = pointcloud + noise
        return  noisy_pointcloud
        
class ToTensor(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        return torch.from_numpy(pointcloud)
    
def default_transforms():
    return transforms.Compose([ Normalize(),
                                ToTensor()
                              ])

In [0]:
def calculateNearest(firstEmbed,secEmbed):
  correctIndex = []
  predictedIndex = []
  difference = []
  for i in range(len(firstEmbed)):
      distance = (secEmbed-firstEmbed[i]).pow(2).sum(1)
      idx, val= torch.argmax(distance),torch.min(distance)
      correctIndex.append(i)
      predictedIndex.append(idx)
      difference.append(val)
      return correctIndex,predictedIndex,difference

In [20]:
from PointNet2 import Net
from embeddings import Encoder
import random

train_split =  int(0.8 * len(modelIdList))
val_split = int(0.9 * len(modelIdList))
train_ds = ShapeText(encoder_input_sentences[ :train_split,:], modelIdList[ :train_split], categoryList[ :train_split], transform=default_transforms())
val_ds = ShapeText(encoder_input_sentences[train_split:val_split,:], modelIdList[train_split:val_split],
                     categoryList[train_split:val_split], train = False, transform=default_transforms())
test_ds = ShapeText(encoder_input_sentences[val_split: ,:], modelIdList[val_split: ], categoryList[val_split: ],
                    train = False, transform=default_transforms())
train_loader = DataLoader(dataset=train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(dataset=train_ds, batch_size=256, shuffle=False)
test_loader = DataLoader(dataset=test_ds, batch_size=256, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
HIDDEN_SIZE = 300
criterion = TripletLoss()
shape_model = Net().to(device)
text_model = Encoder(embedding_matrix, HIDDEN_SIZE).to(device)
params = list(shape_model.parameters()) + list(text_model.parameters())
optimizer = torch.optim.Adam(params, lr=0.001)
shape_model.train()
text_model.train()
train_loss_history = []
train_acc_history = []
val_acc_history = []
val_loss_history = []
num_epochs = 2
iter_per_epoch = len(train_loader)
log_nth = 5
for epoch in range(num_epochs):
    running_loss = []
    for i, (anchor_shape, pos_text, neg_text) in enumerate(train_loader):
        pos_text = pos_text.to(device)
        neg_text = neg_text.to(device)
        
        optimizer.zero_grad()
        anchor_out = shape_model(anchor_shape)
        positive_out = text_model(pos_text)[:,-1,:].squeeze(1)
        negative_out = text_model(neg_text)[:,-1,:].squeeze(1)
        loss = criterion(anchor_out, positive_out, negative_out)
        loss.backward()
        optimizer.step()
        
        if (i+1) % log_nth == 0:
            
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                    .format(epoch+1, num_epochs, i+1, iter_per_epoch, loss.item()))
            
        if (i+1) % iter_per_epoch == 0: 
            correctText,predictedText,difference = calculateNearest(anchor_out, positive_out)
            correctShape,predictedShape,difference = calculateNearest(positive_out, anchor_out)
            total += correctShape.size(0)
            correctText += (correctText == predictedText).sum().item()
            correctShape += (correctShape == predictedShape).sum().item()
            shapeAcc = 100 * (correctShape / total)
            textAcc = 100 * (correctText / total)
            print('Train: Text to Shape Retrieval accuracy: {} %'.format(shapeAcc) )
            print('Train: Shape to Text Retrieval accuracy: {} %'.format(textAcc) )
            train_acc_shape_history.append(shapeAcc)
            train_acc_text_history.append(textAcc)

        # Validate the model
    with torch.no_grad():
        correctText = 0
        correctShape = 0
        total = 0
        for shapes, text in val_loader:
            text,shape = text.to(device),shape.to(device)
            text_embed = text_model(text)[:,-1,:].squeeze(1)
            shape_embed = shape_model(shape)
            correctText,predictedText,difference = calculateNearest(shape_embed, text_embed)
            correctShape,predictedShape,difference = calculateNearest(text_embed, shape_embed)
            total += correctShape.size(0)
            correctText += (correctText == predictedText).sum().item()
            correctShape += (correctShape == predictedShape).sum().item()

        shapeAcc = 100 * (correctShape / total)
        textAcc = 100 * (correctText / total)
        print('Validate: Text to Shape Retrieval accuracy: {} %'.format(shapeAcc) )
        print('Validate: Shape to Text Retrieval accuracy: {} %'.format(textAcc) )
        val_acc_shape_history.append(shapeAcc)
        val_acc_text_history.append(textAcc)

anchor_out: torch.Size([32, 300]) positive_out: torch.Size([32, 300]) negative_out torch.Size([32, 300])
anchor_out: torch.Size([32, 300]) positive_out: torch.Size([32, 300]) negative_out torch.Size([32, 300])
anchor_out: torch.Size([32, 300]) positive_out: torch.Size([32, 300]) negative_out torch.Size([32, 300])
anchor_out: torch.Size([32, 300]) positive_out: torch.Size([32, 300]) negative_out torch.Size([32, 300])
anchor_out: torch.Size([32, 300]) positive_out: torch.Size([32, 300]) negative_out torch.Size([32, 300])
anchor_out: torch.Size([32, 300]) positive_out: torch.Size([32, 300]) negative_out torch.Size([32, 300])


KeyboardInterrupt: ignored

In [11]:
with torch.no_grad():
    correctText = 0
    correctShape = 0
    total = 0
    for shapes, text in test_loader:
        text,shape = text.to(device),shape.to(device)
        text_embed = text_model(text)[:,-1,:].squeeze(1)
        shape_embed = shape_model(shape)
        correctText,predictedText,difference = calculateNearest(shape_embed, text_embed)
        correctShape,predictedShape,difference = calculateNearest(text_embed, shape_embed)
        total += correctShape.size(0)
        correctText += (correctText == predictedText).sum().item()
        correctShape += (correctShape == predictedShape).sum().item()

    shapeAcc = 100 * (correctShape / total)
    textAcc = 100 * (correctText / total)
    print('Test: Text to Shape Retrieval accuracy: {} %'.format(shapeAcc) )
    print('Tesy: Shape to Text Retrieval accuracy: {} %'.format(textAcc) )

<class 'numpy.ndarray'>


In [21]:
from torch import nn
import torch
rnn = nn.LSTM(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, (hn, cn) = rnn(input, (h0, c0))
print("output:",output.shape,"hn",hn.shape,"cn",cn.shape)
print(rnn)

output: torch.Size([5, 3, 20]) hn torch.Size([2, 3, 20]) cn torch.Size([2, 3, 20])
LSTM(10, 20, num_layers=2)


In [12]:
import numpy as np
import torch
x1 = torch.ones((10,2))
x2 = torch.zeros((1,2))
c = (x1 - x2).pow(2).sum(1)
c[0]=c[0]+1
print(torch.argmax(c))

print(torch.max(c))

tensor(0)
tensor(3.)
